## MLflow's Model Registry

In [12]:
from mlflow.tracking import MlflowClient

# Definido a URI para que possamos acessar os artefatos dos modelos:
# MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [13]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# Checando os experimentos presentes no MLFlow:
# client.list_experiments() # -> essa função não é mais utilizada atualmente
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1719876296465, experiment_id='1', last_update_time=1719876296465, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1719800545243, experiment_id='0', last_update_time=1719800545243, lifecycle_stage='active', name='Default', tags={}>]

In [16]:
# Criando um novo experimento no MLFlow (receberá o ID '2'):
client.create_experiment(name="my-cool-experiment")

RestException: RESOURCE_ALREADY_EXISTS: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-cool-experiment', '', 'active', 1720313272230, 1720313272230)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

Let's check the latest versions for the experiment with id `1`...

In [17]:
from mlflow.entities import ViewType # Necessário para o parâmetro 'run_view_type'

# Buscando as melhores runs para o experimento com ID '1' (nyc-taxi-experiment):
runs = client.search_runs(
    experiment_ids='1', # ID do experimento
    filter_string="metrics.rmse < 7", # Filtrando as runs que possuam rmse inferior a 7
    run_view_type=ViewType.ACTIVE_ONLY, # Considerando apenas runs ativas
    max_results=5, # Número máximo de resultados apresentado
    order_by=["metrics.rmse ASC"] # Ordenando os resultados de forma crescente ao RMSE
)

In [18]:
# Apresentando as runs e seus respectivos RMSE's
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 8fbff5ccc37f4eb88fccbeca69dc2da9, rmse: 6.3184
run id: 9dd3fb6b8c134e4dbadec7f32e0378a6, rmse: 6.3198
run id: e0b46ac08d2344c2a1f9cbeddccc166d, rmse: 6.3241
run id: 9d9fd6e804aa4eac8bcfbad11a91721c, rmse: 6.3860
run id: cd1e4ecc43e54004952c8e0d0c3f6623, rmse: 6.4040


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [19]:
import mlflow

# Definindo o URI para o MLFlow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [20]:
# Escolhendo o melhor modelo (menor RMSE) dentre os obtidos anteriormente e o registrando
run_id = "9dd3fb6b8c134e4dbadec7f32e0378a6" # "b8904012c84343b5bf8ee72aa8f0f402"
model_uri = f"runs:/{run_id}/model" # Escolhendo o arquivo do modelo de interesse
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor") # Registrando o modelo com nome 'nyc-taxi-regressor' (encontrado na aba 'Models', 'Registred Models')

Successfully registered model 'nyc-taxi-regressor'.
2024/07/07 01:15:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 1
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1720314934160, current_stage='None', description='', last_updated_timestamp=1720314934160, name='nyc-taxi-regressor', run_id='8fbff5ccc37f4eb88fccbeca69dc2da9', run_link='', source='mlflow-artifacts:/1/8fbff5ccc37f4eb88fccbeca69dc2da9/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [21]:
# Verificando as versões registradas para o modelo "nyc-taxi-regressor"
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/tmp/ipykernel_11652/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [24]:
# Fazendo a transição do modelo registrado na versão 1 para 'Staging'
model_version = 1 # 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_11652/4006274699.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1720314934160, current_stage='Staging', description='', last_updated_timestamp=1720315276834, name='nyc-taxi-regressor', run_id='8fbff5ccc37f4eb88fccbeca69dc2da9', run_link='', source='mlflow-artifacts:/1/8fbff5ccc37f4eb88fccbeca69dc2da9/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [25]:
from datetime import datetime

# Adicionando uma descrição ao modelo de interesse (Versão 1) com a data de mudança
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1720314934160, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-07-07', last_updated_timestamp=1720315565098, name='nyc-taxi-regressor', run_id='8fbff5ccc37f4eb88fccbeca69dc2da9', run_link='', source='mlflow-artifacts:/1/8fbff5ccc37f4eb88fccbeca69dc2da9/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [28]:
from sklearn.metrics import mean_squared_error
import pandas as pd

# Funções utilizadas para o carregamento de dados, preprocessamento de dados e treinamento do modelo
def read_dataframe(filename):
    df = pd.read_parquet(filename) # Adaptando a função para ler um arquivo do tipo 'parquet'

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}") # Carregando o modelo a partir de uma função do Python
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [29]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [30]:
# Baixando o 'preprocessor' para o modelo
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/workspaces/mlops-zoomcamp/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [31]:
import pickle

# Carregando o 'preprocessor' do modelo
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [32]:
# Aplicando o preprocessamento dos dados
X_test = preprocess(df, dv)

In [33]:
# Separando a variável resposta de interesse
target = "duration"
y_test = df[target].values

In [34]:
# Testando o modelo registrando em 'Production' e obtendo seu RMSE (retornará erro pois não terá nenhum modelo)
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/home/codespace/.python/current/lib/python3.10/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


MlflowException: No versions of model with name 'nyc-taxi-regressor' and stage 'Production' found

In [39]:
# Testando o modelo registrando em 'Staging' e obtendo seu RMSE
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/home/codespace/.python/current/lib/python3.10/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])

MlflowException: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/1/8fbff5ccc37f4eb88fccbeca69dc2da9/artifacts/model:
##### File  #####
API request to http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/1/8fbff5ccc37f4eb88fccbeca69dc2da9/artifacts/model/ failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/1/8fbff5ccc37f4eb88fccbeca69dc2da9/artifacts/model/ (Caused by ResponseError('too many 500 error responses'))

In [37]:
# Exemplo de como podemos transisionar um modelo que estava em 'Staging' para Produção'
client.transition_model_version_stage(
    name=model_name,
    version=1, #4
    stage="Production",
    archive_existing_versions=True # Caso existam modelos em Produção, arquivamo-os (espera-se que se coloque um modelo com uma melhor métrica em prod.)
)

/tmp/ipykernel_11652/4125428703.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1720314934160, current_stage='Production', description='The model version 1 was transitioned to Staging on 2024-07-07', last_updated_timestamp=1720316294525, name='nyc-taxi-regressor', run_id='8fbff5ccc37f4eb88fccbeca69dc2da9', run_link='', source='mlflow-artifacts:/1/8fbff5ccc37f4eb88fccbeca69dc2da9/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>